In [45]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

D:\Users\Ramiro\Anaconda3\lib\site-packages\wheel\pep425tags.py:75: RuntimeWarning: Config variable 'Py_DEBUG' is unset, Python ABI tag may be incorrect
  warn=(impl == 'cp')):
D:\Users\Ramiro\Anaconda3\lib\site-packages\wheel\pep425tags.py:79: RuntimeWarning: Config variable 'WITH_PYMALLOC' is unset, Python ABI tag may be incorrect
  warn=(impl == 'cp')):
torch-0.4.1-cp36-cp36m-linux_x86_64.whl is not a supported wheel on this platform.


In [201]:
import torch

cuda = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import numpy as np

totalbars=32 #total bars as input
NUM_PITCHES=4 # all possible notes to play


NOTESPERBAR=16
TOTAL_NOTES=NOTESPERBAR*totalbars

num_features=NUM_PITCHES #size of input feature vector

teacherforcing=False #not used but it will be needed


In [202]:

batch_size=1



#Generating dummy data
a= np.random.randint(NUM_PITCHES, size=TOTAL_NOTES)
print(a)
data = np.zeros((TOTAL_NOTES, NUM_PITCHES))
data[np.arange(TOTAL_NOTES), a] = 1 #generating dummy data


#adding 1 dimension, 1x32x4
data = data[np.newaxis, :, :]

data.shape

[0 0 0 3 1 3 1 2 3 3 2 0 3 1 2 3 0 2 3 3 2 2 2 0 0 2 3 2 0 3 2 2 2 2 2 1 1
 3 0 0 1 3 0 3 3 0 2 1 0 0 3 3 1 0 0 1 3 3 2 2 1 3 1 0 0 0 3 1 3 1 3 1 0 1
 3 0 1 3 2 0 3 0 0 1 1 3 3 0 3 1 3 3 1 1 1 1 0 3 2 0 2 0 3 0 2 3 1 0 3 0 1
 2 0 3 0 2 1 1 1 0 3 3 0 0 0 1 2 3 1 2 1 0 3 1 3 2 1 2 3 0 2 0 0 1 0 2 2 2
 3 0 2 2 1 3 1 0 1 0 0 0 1 2 3 2 2 2 2 0 1 3 0 1 2 0 3 2 0 0 1 0 2 3 2 0 1
 0 3 3 3 0 1 1 2 3 1 0 1 2 1 2 3 2 3 1 0 2 1 3 2 3 2 3 0 3 3 0 3 1 1 2 3 2
 2 0 1 0 1 0 0 3 3 2 1 1 1 0 1 3 2 2 2 1 0 2 2 1 2 2 3 2 0 1 0 2 0 2 2 1 3
 3 3 3 3 3 1 3 2 2 2 0 0 2 3 2 0 0 0 2 2 3 1 2 2 1 1 2 0 2 0 2 2 0 0 3 0 1
 3 2 3 1 2 3 0 1 2 1 1 2 1 2 0 2 1 0 0 1 1 3 2 1 0 1 0 0 2 1 3 1 2 0 1 0 1
 1 3 0 0 2 1 3 3 0 1 3 1 0 3 1 1 2 1 1 3 0 1 3 3 0 0 2 1 0 3 3 0 1 0 1 3 2
 3 2 0 3 1 3 0 1 1 0 3 2 0 1 0 2 1 2 1 0 3 3 0 0 3 0 2 3 2 0 1 1 0 2 3 0 2
 3 0 2 1 1 2 0 1 3 1 0 0 1 1 2 0 3 2 3 3 2 3 2 1 2 2 0 1 3 3 1 3 1 2 2 1 2
 3 3 2 0 2 3 2 0 3 0 2 1 1 3 0 3 1 2 3 0 0 2 1 0 0 3 2 1 0 0 1 2 2 1 3 0 0
 3 0 2 0 1 3 1 2 1 0 0 2 

(1, 512, 4)

In [203]:
#half of this is not yet needed but maybe it will be to visualize the latent space
import matplotlib.pyplot as plt
from IPython.display import Image, display, clear_output
import numpy as np
from sklearn.decomposition import PCA
%matplotlib nbagg
%matplotlib inline
import seaborn as sns
sns.set_style("whitegrid")
sns.set_palette(sns.dark_palette("purple"))

MODEL:

In [206]:
import torch.nn as nn
from torch.nn.functional import softplus

# define size variables
input_size = NUM_PITCHES

enc_hidden_size=2048 #hidden size of encoder
conductor_hidden_size=1024 #hidden size of decoder


decoders_hidden_size=1024 #hidden size of decoder
decoders_initial_size=512 #decoder input size

n_embeddings=16

n_layers_conductor=2
n_layers_decoder=3


latent_features=512 #latent space dimension



class VariationalAutoencoder(nn.Module):
    def __init__(self, latent_features):
        super(VariationalAutoencoder, self).__init__()
        
        self.latent_features = latent_features
       
        
        
        #data goes into bidirectional encoder
        self.encoder = torch.nn.LSTM(
                batch_first = True,
                input_size = input_size,
                hidden_size = enc_hidden_size,
                num_layers = 1,
                bidirectional = True)
        
        #encoded data goes onto connect linear layer. inputs must be*2 because LSTM is bidirectional
        #output must be 2*latentspace because it needs to be split into miu and sigma right after.
        self.encoderOut = nn.Linear(in_features=enc_hidden_size*2, out_features=latent_features*2)
        
        
        #Model after this line has not been tested -------------
        self.decoderIn = nn.Linear(in_features=latent_features, out_features=conductor_initial_size)
        
        # The latent code must be decoded into the original image
        self.conductor = torch.nn.LSTM(
                batch_first=True,
                input_size=self.latent_features,
                hidden_size=conductor_hidden_size,
                num_layers=2,
                bidirectional = False)
                
                #Model after this line has not been tested -------------
        self.conductorout = nn.Linear(in_features=TOTAL_NOTES, out_features=2*n_embeddings)    
            
        # The latent code must be decoded into the original image
        self.decoders = [torch.nn.LSTM(
                batch_first=True,
                input_size=decoders_initial_size,
                hidden_size=decoders_hidden_size,
                num_layers=2,
                bidirectional = False)] * n_embeddings

    #used to initialize the hidden layer of the encoder to zero before every batch
    def init_hidden(self, batch_size):
      #must be 2 x batch x hidden_size because its a bi-directional LSTM
      init = torch.zeros(2, batch_size, enc_hidden_size, device=device)
      c0 = torch.zeros(2, batch_size, enc_hidden_size, device=device)
    
      #2 because has 2 layers
      init_conductor = torch.zeros(n_layers_conductor, batch_size, conductor_hidden_size, device=device)
      c_condunctor = torch.zeros(n_layers_conductor, batch_size, conductor_hidden_size, device=device)
    
      #2 because has 2 layers
      init_decoders = torch.zeros(n_layers_decoder, batch_size, decoders_hidden_size, device=device)
      c_decoders= torch.zeros(n_layers_decoder, batch_size, decoders_hidden_size, device=device)
        
        
      return init,c0,init_conductor,c_condunctor,init_decoders,c_decoders


    def forward(self, x): 
        outputs = {}
        
        print("INPUT SIZE",x.size())
        
        #creates hidden layer values
        h0,c0,hconductor,cconductor,hdecoders,cdecoders = self.init_hidden(batch_size)
        
        #resets encoder at the beginning of every batch
        x, hidden = self.encoder(x, ( h0,c0))
          
        print("after rnn SIZE",x.shape)
        
        #goes from 4096 to 1024
        x = self.encoderOut(x)
        
        print("after linear",x.shape)
        
        # Split encoder outputs into a mean and variance vector
        mu, log_var = torch.chunk(x, 2, dim=-1)
        
        print("MU SIZE",mu.shape)
        print("LOG SIZE",log_var.shape)
        
        # Make sure that the log variance is positive
        log_var = softplus(log_var)
        
       
        # :- Reparametrisation trick
        # a sample from N(mu, sigma) is mu + sigma * epsilon
        # where epsilon ~ N(0, 1)
                
        # Don't propagate gradients through randomness
        with torch.no_grad():
            batch_size = mu.size(0)
            epsilon = torch.randn(batch_size, 1, self.latent_features)
            
            if cuda:
                epsilon = epsilon.cuda()
        
        #setting sigma
        sigma = torch.exp(log_var*2)
        
        
        print("epsilon",epsilon.shape)
        
        print("ready to make latent space",sigma.shape)
        
        #generate z
        z = mu + epsilon * sigma        
        
        print("z space generated",z.shape)
        
        
        #NOT DONE AFTER THIS
        
        # Run through decoder
        x, hidden = self.conductor(z, ( hconductor,cconductor))
        
        print("conductor",x.shape)
        
        x = x.permute(0, 2, 1)
        
        x = self.conductorout(x)
        
        x = x.permute(0, 2, 1)
        
        
        print("conductor after lin",x.shape)
        
        mu_embeddings, log_var_embeddings = torch.chunk(x, 2, dim=-2)
        
        
        
        
        # Make sure that the log variance is positive
        log_var_embeddings = softplus(log_var_embeddings)
        
        # :- Reparametrisation trick
        # a sample from N(mu, sigma) is mu + sigma * epsilon
        # where epsilon ~ N(0, 1)
                
        # Don't propagate gradients through randomness
        with torch.no_grad():
            
            epsilon_embeddings = torch.randn(mu.size(0), 1, conductor_hidden_size)
            
            if cuda:
                epsilon_embeddings = epsilon_embeddings.cuda()
        
        
        
        
        sigma_embeddings = torch.exp(log_var_embeddings/2)
        
        print("generated sigma_embeddings",sigma_embeddings.shape)
        print("generated log_var_embeddings",log_var_embeddings.shape)
        
        print("epsilon_embeddings",epsilon_embeddings.shape)
        
        #generate embeddings
        embeddings = mu_embeddings + epsilon_embeddings * sigma_embeddings     
        
        outputs["embeddings"] = embeddings
        
        print("generated embeddings",embeddings.shape)
        
        
        
        for c in range(embeddings.shape[1]):
            
            
            if c == 0:
                inputs = torch.cat((embeddings[:,c,:],torch.zeros(batch_size,32,4)), 1)
            else:
                inputs = torch.cat((embeddings[:,c,:],twobar), 1)
            
            
            
            
            twobar, hidden = self.decoders[c](inputs, (hdecoders,cdecoders))
            
    
        
        
        
        
        
        
        
        
        
        
        
      
        
        # The original digits are on the scale [0, 1]
        x = torch.sigmoid(x)
        
        # Mean over samples
        x_hat = torch.mean(x, dim=1)
        
        outputs["x_hat"] = x_hat
        outputs["z"] = z
        outputs["mu"] = mu
        outputs["log_var"] = log_var
        
        return outputs




net = VariationalAutoencoder(latent_features)

# Transfer model to GPU if available
if cuda:
    net = net.cuda()

print(net)

VariationalAutoencoder(
  (encoder): LSTM(4, 2048, batch_first=True, bidirectional=True)
  (encoderOut): Linear(in_features=4096, out_features=1024, bias=True)
  (decoderIn): Linear(in_features=512, out_features=512, bias=True)
  (conductor): LSTM(512, 1024, num_layers=2, batch_first=True)
  (conductorout): Linear(in_features=512, out_features=32, bias=True)
)


Optimizer

In [171]:
#directly taken from notebook, probably some adaptation might be needed

from torch.nn.functional import binary_cross_entropy
from torch import optim



def ELBO_loss(y, t, mu, log_var):
    # Reconstruction error, log[p(x|z)]
    # Sum over features
    likelihood = -binary_cross_entropy(y, t, reduction="none")
    likelihood = likelihood.view(likelihood.size(0), -1).sum(1)

    # Regularization error: 
    # Kulback-Leibler divergence between approximate posterior, q(z|x)
    # and prior p(z) = N(z | mu, sigma*I).
    
    # In the case of the KL-divergence between diagonal covariance Gaussian and 
    # a standard Gaussian, an analytic solution exists. Using this excerts a lower
    # variance estimator of KL(q||p)
    kl = -0.5 * torch.sum(1 + log_var - mu**2 - torch.exp(log_var), dim=1)

    # Combining the two terms in the evidence lower bound objective (ELBO) 
    # mean over batch
    ELBO = torch.mean(likelihood) - torch.mean(kl)
    
    # notice minus sign as we want to maximise ELBO
    return -ELBO, kl.sum()



# define our optimizer
# The Adam optimizer works really well with VAEs.
optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = ELBO_loss

Testing if forward pass works

In [172]:
from torch.autograd import Variable

#setting dummy data
x = data



#setting input ans tensor variable
x = Variable(torch.Tensor(x))

if cuda:
    x = x.cuda()
    
    

#running forward pass
outputs = net(x)


#AFTER THIS NOTHING IS DONE YET ------
x_hat = outputs["x_hat"]
mu, log_var = outputs["mu"], outputs["log_var"]
z = outputs["z"]

loss, kl = loss_function(x_hat, x, mu, log_var)

print(x.shape)
print(x_hat.shape)
print(z.shape)
print(loss)



                
                
        
print(kl)

INPUT SIZE torch.Size([1, 512, 4])
after rnn SIZE torch.Size([1, 512, 4096])
after linear torch.Size([1, 512, 1024])
MU SIZE torch.Size([1, 512, 512])
LOG SIZE torch.Size([1, 512, 512])
epsilon torch.Size([1, 1, 512])
ready to make latent space torch.Size([1, 512, 512])
z space generated torch.Size([1, 512, 512])
conductor torch.Size([1, 512, 1024])
conductor after lin torch.Size([1, 32, 1024])
generated sigma_embeddings torch.Size([1, 16, 1024])
generated log_var_embeddings torch.Size([1, 16, 1024])
epsilon_embeddings torch.Size([1, 1, 1024])
generated embeddings torch.Size([1, 16, 1024])


UnboundLocalError: local variable 'counter' referenced before assignment

In [125]:
net.decoders

[LSTM(512, 1024, num_layers=2, batch_first=True),
 LSTM(512, 1024, num_layers=2, batch_first=True),
 LSTM(512, 1024, num_layers=2, batch_first=True),
 LSTM(512, 1024, num_layers=2, batch_first=True),
 LSTM(512, 1024, num_layers=2, batch_first=True),
 LSTM(512, 1024, num_layers=2, batch_first=True),
 LSTM(512, 1024, num_layers=2, batch_first=True),
 LSTM(512, 1024, num_layers=2, batch_first=True),
 LSTM(512, 1024, num_layers=2, batch_first=True),
 LSTM(512, 1024, num_layers=2, batch_first=True),
 LSTM(512, 1024, num_layers=2, batch_first=True),
 LSTM(512, 1024, num_layers=2, batch_first=True),
 LSTM(512, 1024, num_layers=2, batch_first=True),
 LSTM(512, 1024, num_layers=2, batch_first=True),
 LSTM(512, 1024, num_layers=2, batch_first=True),
 LSTM(512, 1024, num_layers=2, batch_first=True)]

In [126]:
log_var.shape


torch.Size([1, 512, 512])

In [128]:
outputs["embeddings"].shape

torch.Size([1, 256, 1024])

In [143]:
 outputs["embeddings"] =outputs["embeddings"].permute(0, 2, 1)

In [144]:
outputs["embeddings"].shape

torch.Size([1, 1024, 256])

In [151]:
ola = outputs["embeddings"]

In [153]:
#Generating dummy data
a= np.random.randint(NUM_PITCHES, size=TOTAL_NOTES)
print(a)
data = np.zeros((TOTAL_NOTES, NUM_PITCHES))
data[np.arange(TOTAL_NOTES), a] = 1 #generating dummy data
data.shape

[3 2 3 1 1 2 3 0 1 2 2 1 0 1 2 2 3 2 0 3 0 3 1 0 2 0 2 3 0 2 0 2 3 1 1 2 0
 3 0 1 1 1 0 2 3 0 0 1 1 1 2 2 2 0 1 0 1 3 0 3 1 0 3 2 0 3 1 3 3 1 0 3 2 1
 3 0 3 2 3 1 3 0 3 2 0 1 3 2 0 3 1 0 2 1 0 2 3 2 3 0 1 0 0 1 2 0 2 1 2 2 1
 2 3 2 1 2 0 0 1 3 2 0 2 0 0 3 2 1 1 0 1 1 2 1 3 1 0 1 2 2 3 1 3 1 1 3 2 2
 1 1 0 2 1 3 1 0 1 0 1 0 0 0 0 2 3 1 2 1 2 2 1 0 1 0 1 1 2 2 0 0 0 2 0 3 1
 2 1 1 0 1 2 1 3 0 3 3 0 3 0 0 0 3 2 0 0 2 0 0 1 1 3 0 3 1 0 1 3 1 0 2 1 1
 2 1 3 0 2 0 2 3 3 0 3 2 0 3 3 3 3 2 3 0 2 2 3 3 1 3 2 1 2 2 0 1 0 0 0 2 2
 3 1 0 3 2 1 1 3 0 0 2 3 0 1 3 1 3 2 3 0 1 0 2 2 3 0 2 1 1 1 2 1 1 2 1 2 3
 2 1 0 0 2 2 0 3 1 0 0 2 1 0 2 3 1 1 1 3 1 0 1 2 1 1 1 3 1 3 2 3 0 3 3 1 2
 3 0 0 1 0 0 0 1 3 3 0 3 2 1 2 1 3 0 1 1 3 0 2 3 0 1 1 1 1 0 1 3 0 1 0 2 2
 3 2 0 1 2 3 0 3 3 3 0 3 0 0 2 1 0 1 1 3 1 3 3 0 3 0 2 1 1 3 1 1 1 2 1 0 2
 0 1 2 0 1 2 2 2 2 2 0 0 0 0 1 1 3 1 2 2 2 3 1 0 1 0 2 3 2 0 1 0 0 0 3 2 0
 3 0 2 1 0 0 0 2 1 2 3 3 2 3 3 0 1 3 1 2 0 1 3 2 2 1 2 1 3 1 2 2 2 3 1 2 3
 0 3 2 2 1 0 2 2 2 2 0 1 

(512, 4)

In [181]:
ola = outputs["embeddings"]

counter=0

    
    
    
print(counter)


0


In [166]:
ola.shape

torch.Size([1, 16, 1024])

In [174]:
        net.decoders[0](x)

RuntimeError: input.size(-1) must be equal to input_size. Expected 512, got 4

In [191]:
counter=0

for c in range(ola.shape[1]):
            
        print(ola[:,c,:])
        counter=counter+1
            
print(counter)
           

tensor([[ 1.5360, -1.3293,  1.4544,  ..., -0.4645, -1.2291,  1.5313]],
       device='cuda:0', grad_fn=<SelectBackward>)
tensor([[ 1.4671, -1.5170,  1.2999,  ..., -0.7309, -1.1208,  1.5104]],
       device='cuda:0', grad_fn=<SelectBackward>)
tensor([[ 1.4940, -1.3939,  1.4040,  ..., -0.5393, -1.2497,  1.4870]],
       device='cuda:0', grad_fn=<SelectBackward>)
tensor([[ 1.4695, -1.4243,  1.2344,  ..., -0.6600, -1.0983,  1.5696]],
       device='cuda:0', grad_fn=<SelectBackward>)
tensor([[ 1.4490, -1.4882,  1.2845,  ..., -0.6755, -1.2087,  1.4911]],
       device='cuda:0', grad_fn=<SelectBackward>)
tensor([[ 1.4950, -1.4606,  1.4624,  ..., -0.5807, -1.2787,  1.4501]],
       device='cuda:0', grad_fn=<SelectBackward>)
tensor([[ 1.5577, -1.3363,  1.4332,  ..., -0.4884, -1.1859,  1.5721]],
       device='cuda:0', grad_fn=<SelectBackward>)
tensor([[ 1.5388, -1.4286,  1.4066,  ..., -0.5841, -1.2109,  1.5612]],
       device='cuda:0', grad_fn=<SelectBackward>)
tensor([[ 1.5366, -1.4085,  1.33

In [184]:
ola[].shape


IndexError: index 1 is out of bounds for dimension 0 with size 1

In [190]:
ola[:,1,:]

tensor([[ 1.4671, -1.5170,  1.2999,  ..., -0.7309, -1.1208,  1.5104]],
       device='cuda:0', grad_fn=<SelectBackward>)

In [207]:


torch.zeros(batch_size,4,4)

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])